In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BI/Global_Superstore2.csv', encoding='latin1')
df.head(5)


In [ ]:
df.info()
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop("Postal Code",axis=1,inplace=True)

In [ ]:
def check_duplicates(df):
    print("##################### Duplicates #####################")
    print(df.duplicated().sum())
check_duplicates(df)

In [ ]:
import numpy as np
# show the missing values in dataset with ratio
def missing_values_tabl(df):

    na_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (df[na_columns].isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio,2)], axis=1, keys=['n_miss', 'ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

missing_values_tabl(df)

In [ ]:
# converting object to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['Ship Date']  = pd.to_datetime(df['Ship Date'],  dayfirst=True, errors='coerce')

In [ ]:
# creating new features
df["order year"]=df["Order Date"].dt.year
df["order_month"]=df["Order Date"].dt.month
df["ship year"]=df["Ship Date"].dt.year
df["ship_month"]=df["Ship Date"].dt.month
# unit price
df["unit_price"]=df["Sales"]/df["Quantity"]
# rename columns
df.rename(columns={"Order Date":"order_date","Ship Date":"ship_date","Order Priority":"order_priority"},inplace=True)

# **K-Means Clustering ( Phân cụm khách hàng ) **

In [ ]:
# Tính toán RFM cho phân cụm
rfm = df.groupby('Customer ID').agg({
    'order_date': lambda x: (df['order_date'].max() - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
}).reset_index()
rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

In [ ]:
# Kiểm tra xem rfm có NaN không
print(rfm[['Recency','Frequency','Monetary']].isnull().sum())

In [ ]:
# Tạo DataFrame sạch: drop những dòng có Recency NaN
rfm = rfm.dropna(subset=['Recency','Frequency','Monetary']).reset_index(drop=True)
# Chuẩn hóa dữ liệu khách hàng
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency','Frequency','Monetary']])

In [ ]:
# Tính SSE cho Elbow Method
sse = []
for k in range(1, 11):
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(rfm_scaled)
    sse.append(km.inertia_)

plt.figure(figsize=(6,4))
plt.plot(range(1,11), sse, 'o-')
plt.xlabel('k clusters')
plt.ylabel('SSE')
plt.title('Elbow method cho RFM')
plt.grid(True)
plt.show()


In [ ]:
sil_scores = []
for k in range(2,11):
    km = KMeans(n_clusters=k, random_state=1).fit(rfm_scaled)
    sil_scores.append(silhouette_score(rfm_scaled, km.labels_))
plt.figure()
plt.plot(range(2,11), sil_scores, 'o-')
plt.xlabel('k clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis cho RFM')
plt.show()


In [ ]:
for k, (inertia, sil) in enumerate(zip(sse[1:], sil_scores), start=2):
    print(f"k={k}: inertia={inertia:.2f}, silhouette={sil:.3f}")

In [ ]:
# Áp dụng KMeans với k tối ưu
k_optimal = 3
kmeans = KMeans(n_clusters=k_optimal, random_state=1)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

In [ ]:
# Phân tích kết quả KMeans
print("Silhouette Score KMeans:", silhouette_score(rfm_scaled, rfm['Cluster']))

In [ ]:
# Đếm số lượng khách hàng trong mỗi cụm
counts = rfm['Cluster'].value_counts().sort_index()
# Phân tích kết quả trung bình các đặc trưng theo cụm
cluster_summary = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean().round(2)
print("\n Trung bình RFM theo từng cụm:\n", cluster_summary)


In [ ]:
# Vẽ biểu đồ trực quan
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pcs = pca.fit_transform(rfm_scaled)
df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
df_pca['Cluster'] = rfm['Cluster']
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='Cluster', palette='tab10')
plt.title("RFM clusters in 2D PCA space")
plt.show()

sns.pairplot(rfm, hue='Cluster', vars=['Recency', 'Frequency', 'Monetary'], palette='tab10')
plt.suptitle("Phân cụm khách hàng theo RFM", y=1.02)
plt.show()

# **K-Means Clustering ( Phân cụm sản phẩm ) **

In [ ]:
# Tạo feature cho sản phẩm
product_df = (
    df.groupby('Product Name')
      .agg(
         OrderCount   = ('Order ID', 'nunique'),
         TotalQuantity= ('Quantity', 'sum'),
         TotalSales   = ('Sales', 'sum'),
         TotalProfit  = ('Profit', 'sum'),
         AvgPrice     = ('unit_price', 'mean')
      )
      .reset_index()
)


In [ ]:
# Chuẩn hóa dữ liệu của sản phẩm
X = product_df[['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Tính SSE cho Elbow Method
sse = []
K_range = range(1, 11)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    sse.append(km.inertia_)

# Vẽ Elbow plot
plt.figure(figsize=(8, 4))
plt.plot(K_range, sse, 'o-', linewidth=2)
plt.xlabel('Số cụm k')
plt.ylabel('SSE (Inertia)')
plt.title('Elbow Method – Sản phẩm')
plt.grid(True)
plt.show()

In [ ]:
# Áp dụng K-means với K tối ưu
k_opt_p = 3
product_df['KMeans_Cluster'] = KMeans(n_clusters=k_opt_p, random_state=42).fit_predict(X)
print("Silhouette Score (products):", silhouette_score(X, product_df['KMeans_Cluster']))

In [ ]:
# Thống kê trung bình theo cụm
cluster_summary = product_df.groupby('KMeans_Cluster')[['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice']].mean().round(2)
print(" Trung bình theo cụm sản phẩm:")
print(cluster_summary)

In [ ]:
# Đếm số lượng sản phẩm trong mỗi cụm
product_df['KMeans_Cluster'].value_counts().sort_index()


In [ ]:
# PCA 2D scatter
pca = PCA(n_components=2)
pcs = pca.fit_transform(X_scaled)
df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
df_pca['KMeans_Cluster'] = product_df['KMeans_Cluster']

plt.figure(figsize=(6,4))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='KMeans_Cluster', palette='tab10', s=50)
plt.title("PCA projection of Product Clusters")
plt.show()

# Boxplot TotalProfit theo cụm
plt.figure(figsize=(8,4))
sns.boxplot(data=product_df, x='KMeans_Cluster', y='TotalProfit', palette='Set2')
plt.title("Phân phối TotalProfit theo cụm sản phẩm")
plt.show()

# Pairplot (tuỳ chọn)
sns.pairplot(product_df, hue='KMeans_Cluster', vars=X, palette='tab10')
plt.suptitle("Pairplot - Product Clusters", y=1.02)
plt.show()

# **DBSCAN**

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
# Áp dụng DBSCAN
dbscan = DBSCAN(eps=0.33, min_samples=4)
rfm['DBSCAN_Cluster'] = dbscan.fit_predict(rfm_scaled)

In [ ]:
# Đếm số cụm và nhiễu (-1 là noise)
print("Số lượng cụm:", len(rfm['DBSCAN_Cluster'].unique()) - (1 if -1 in rfm['DBSCAN_Cluster'].unique() else 0))
print("Số khách hàng nhiễu (noise):", (rfm['DBSCAN_Cluster'] == -1).sum())

In [ ]:
# Tính silhouette score (chỉ tính nếu có hơn 1 cụm)
n_clusters = len(set(rfm['DBSCAN_Cluster'])) - (1 if -1 in rfm['DBSCAN_Cluster'].values else 0)
if n_clusters > 1:
    score = silhouette_score(rfm_scaled, rfm['DBSCAN_Cluster'])
    print(f"Silhouette Score: {score:.3f}")
else:
    print("Không thể tính Silhouette Score (chỉ có 1 cụm hoặc toàn outlier).")

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
components = pca.fit_transform(rfm_scaled)
rfm['PCA1'] = components[:,0]
rfm['PCA2'] = components[:,1]

plt.figure(figsize=(10,6))
sns.scatterplot(data=rfm, x='PCA1', y='PCA2', hue='DBSCAN_Cluster', palette='Set1', legend='full')
plt.title("Phân cụm khách hàng bằng DBSCAN (PCA 2D)")
plt.legend(title='Cụm')
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

dbscan = DBSCAN(eps=2.5, min_samples=4)
product_df['DBSCAN_Cluster'] = dbscan.fit_predict(X_scaled)

n_clusters = len(set(product_df['DBSCAN_Cluster'])) - (1 if -1 in product_df['DBSCAN_Cluster'] else 0)
n_noise = sum(product_df['DBSCAN_Cluster'] == -1)

print("Số cụm:", n_clusters)
print("Số sản phẩm nhiễu (noise):", n_noise)

# Tính Silhouette Score nếu đủ cụm
if n_clusters > 1:
    score = silhouette_score(X_scaled[product_df['DBSCAN_Cluster'] != -1],
                             product_df['DBSCAN_Cluster'][product_df['DBSCAN_Cluster'] != -1])
    print(f"Silhouette Score: {score:.3f}")
else:
    print("Không thể tính Silhouette Score.")


In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

pca = PCA(n_components=2)
product_components = pca.fit_transform(X_scaled)

product_df['PCA1'] = product_components[:,0]
product_df['PCA2'] = product_components[:,1]

plt.figure(figsize=(10,6))
sns.scatterplot(data=product_df, x='PCA1', y='PCA2', hue='DBSCAN_Cluster', palette='Set1')
plt.title("Phân cụm sản phẩm bằng DBSCAN (PCA 2D)")
plt.legend(title='Cụm')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

# --- Hàm clustering không tính silhouette, chỉ gán nhãn ---
def apply_kmeans(X_scaled, df, n_clusters, label_name):
    km = KMeans(n_clusters=n_clusters, random_state=42)
    labels = km.fit_predict(X_scaled)
    df[f"{label_name}_KMeans"] = labels

def apply_dbscan(X_scaled, df, eps, min_samples, label_name):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(X_scaled)
    df[f"{label_name}_DBSCAN"] = labels

def plot_pca_comparison(X_scaled, df, label_name):
    pca = PCA(n_components=2)
    pcs = pca.fit_transform(X_scaled)
    df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
    df_pca[label_name+'_KMeans'] = df[f"{label_name}_KMeans"].values
    df_pca[label_name+'_DBSCAN'] = df[f"{label_name}_DBSCAN"].values

    fig, axes = plt.subplots(1,2, figsize=(12,5))
    sns.scatterplot(ax=axes[0],
                    data=df_pca, x='PC1', y='PC2',
                    hue=label_name+'_KMeans', palette='tab10', s=40)
    axes[0].set_title(f"{label_name} – KMeans (k=3)")

    sns.scatterplot(ax=axes[1],
                    data=df_pca, x='PC1', y='PC2',
                    hue=label_name+'_DBSCAN', palette='Set1', s=40)
    axes[1].set_title(f"{label_name} – DBSCAN")
    plt.tight_layout()
    plt.show()

# --- Phân cụm & trực quan hóa cho RFM ---
apply_kmeans(rfm_scaled, rfm, n_clusters=3, label_name='RFM')
apply_dbscan(rfm_scaled, rfm, eps=0.33, min_samples=4, label_name='RFM')
plot_pca_comparison(rfm_scaled, rfm, label_name='RFM')

# --- Phân cụm & trực quan hóa cho Product ---
apply_kmeans(X_scaled, product_df, n_clusters=3, label_name='Prod')
apply_dbscan(X_scaled, product_df, eps=2.5, min_samples=4, label_name='Prod')
plot_pca_comparison(X_scaled, product_df, label_name='Prod')
